In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import re
import sqlite3
import nltk
from nltk.corpus import stopwords
from sklearn.multiclass import OneVsRestClassifier

nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

conn = sqlite3.connect('yt_videos_for_train_labeled.db')
videos_df = pd.read_sql('SELECT * FROM videos', conn)

# Предобработка текста
def clean_text(text):
    # Удаление HTML-тегов, специальных символов и цифр
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

# Конкатенация названия видео и описания
videos_df['text'] = videos_df['video_title'].astype(str) + " " + videos_df['video_description'].astype(str)
videos_df['text'] = videos_df['text'].apply(clean_text)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(videos_df['text'], videos_df['profession'], test_size=0.2, random_state=42)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yurik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Создание конвейера с TF-IDF векторизацией и OneVsRestClassifier с логистической регрессией
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
])

# Предполагаем, что 'y_train' и 'y_test' теперь имеют мультилейбловый формат
pipeline.fit(X_train, y_train)

# Оценка модели
predictions = pipeline.predict(X_test)
print(classification_report(y_test, predictions))

                                                                       precision    recall  f1-score   support

                                                            AI-тренер       0.53      0.47      0.50        57
                                                   Digital-маркетолог       0.46      0.60      0.52        89
                                                 Frontend-разработчик       0.69      0.73      0.71       114
                                                          Автомеханик       0.54      0.83      0.65       127
                                                Агент в сфере туризма       0.69      0.76      0.73       102
                                                              Агроном       0.95      0.96      0.96       106
                                                              Адвокат       0.61      0.71      0.66       100
                                              Администратор ресторана       0.83      0.14      0.24        36


d:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer

# Создание конвейера с TF-IDF векторизацией
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Предобработка данных и векторизация
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Преобразуем метки в двоичный формат
mlb = MultiLabelBinarizer()
y_train_mlb = mlb.fit_transform(y_train)
y_test_mlb = mlb.transform(y_test)

# Преобразование разреженных матриц TF-IDF в плотные матрицы NumPy
X_train_tfidf_dense = X_train_tfidf.toarray()
X_test_tfidf_dense = X_test_tfidf.toarray()

# Создаем модель нейронной сети как раньше
model = Sequential()
model.add(Dense(512, input_shape=(X_train_tfidf_dense.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train_mlb.shape[1], activation='sigmoid'))  # Количество выходных узлов равно количеству классов

# Компилируем модель с binary cross-entropy как раньше
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Обучаем модель, теперь используя плотные матрицы NumPy
model.fit(X_train_tfidf_dense, y_train_mlb, epochs=10, batch_size=32, validation_split=0.1)

# Оцениваем модель на тестовых данных
score = model.evaluate(X_test_tfidf_dense, y_test_mlb, batch_size=32)

# Выводим результаты оценки
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')


ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'scipy.sparse._csr.csr_matrix'>]